In [9]:
import flymovie as fm
import numpy as np
import os
import scipy.ndimage as ndi
from importlib import reload
import scipy
from skimage.measure import regionprops
from flymovie.general_functions import extract_box, stack_normalize_minmax
from copy import deepcopy

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
stack = fm.read_czi('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/psf/2022-02-09/tetraspeck-field1-100x50.czi', swapaxes=False)

def extract_beads(stack, thresh, box_dims):
    """Extract windows centered at beads."""

    # Make mask with thresholding, use opening to get rid of small objects, label.
    mask = np.where(stack > thresh, 1, 0)
    mask = ndi.morphology.binary_opening(mask, structure=np.ones((3,3,3)))

    # Strategy: ndi.measurements.center_of_mass finds the center of mass of objects
    # in a labelmask. I'm worried that if I use the thresholded mask, subtle
    # differences in thresholding might change the center. It would be better
    # to determine the center of mass within a nice window around each object. 
    # To do this, first expand the objects using morphological
    # dilation and find the center of mass of these larger objects.

    mask = ndi.morphology.binary_dilation(mask, structure=np.ones((4,8,8)))
    lmask,_ = ndi.label(mask)
    """
    props = regionprops(lmask)
    for id_ in range(0, len(props)):
        centroid = [int(x) for x in props[id_].centroid]
        sl = ()
        for d in range(len(centroid)):
            halflen = int(box_dims[d] / 2)
            sl = sl + tuple([slice(centroid[d] - halflen, centroid[d] + halflen + 1)])
        lmask[sl] = id_
    """
    # Find center of mass for each spot (within window defined above).
    centers = ndi.measurements.center_of_mass(stack, lmask, np.arange(1, np.max(lmask) + 1))

    boxes = np.ndarray(tuple([0]) + box_dims)
    for center in centers:
            center = [round(x) for x in center]
            box = extract_box(stack, center, box_dims, pad=False)
            if not np.array_equal(box.shape, box_dims):
                continue
            box = stack_normalize_minmax(box)
            box = np.expand_dims(box, axis=0)
            boxes = np.vstack([boxes, box])
    return boxes

boxes = extract_beads(stack, 40, (21,21,21))

In [35]:
fm.viewer(boxes.mean(axis=0) * 1000, 4)

interactive(children=(Dropdown(description='Color', index=5, options=('gray', 'inferno', 'Gators', 'plasma', '…

In [ ]:
stack1 = fm.read_czi('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/psf/2022-02-09/tetraspeck-100x50-field4-1AU-green-01.czi', swapaxes=False)
stack2 = fm.read_czi('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/psf/2022-02-09/tetraspeck-100x50-field5-1AU-green-01.czi', swapaxes=False)

stack = [stack1[0], stack1[1], stack2[0], stack2[1], stack2[2], stack2[3], stack2[4]]

In [5]:
sl = tuple([slice(0,4), slice(0,4)])
np.ones((5,5))[sl]


array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])